이번 챕터에서는 토치텍스트에서 배치퍼스트(batch_first)를 True로 한 경우와 False를 한 경우를 비교해보겠습니다. 이번 챕터는 토치텍스트 튜토리얼 챕터가 아니므로, 토치텍스트를 이전 챕터를 통해 이미 알고 있다고 가정합니다.

### 1. 훈련 데이터와 테스트 데이터로 분리하기

In [1]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import urllib.request
import pandas as pd

In [3]:
# 우선 인터넷에서 IMDB 리뷰 데이터를 다운로드 받습니다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f6977da6df0>)

In [4]:
# 다운로드한 IMDB 리뷰 데이터를 데이터프레임에 저장합니다.
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')

In [5]:
# 전체 샘플의 개수를 보겠습니다.
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [6]:
# 전체 샘플의 개수는 50,000개입니다. 25,000개씩 분리하여 훈련 데이터와 테스트 데이터로 분리해보겠습니다.
train_df = df[:25000]
test_df = df[25000:]

In [7]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)
# index=False를 하면 인덱스를 저장하지 않습니다.

### 2. 필드 정의하기(torchtext.data)

In [8]:
from torchtext import data

# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True, # <== 이 부분을 True로 합니다.
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

### 3. 데이터셋 / 단어 집합 / 데이터로더 만들기

In [9]:
from torchtext.data import TabularDataset
from torchtext.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠습니다.
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치

In [10]:
print(batch.text)

tensor([[  10,   20,  469,  137,  590, 1450,    4,  197,  605,  827,    2,  670,
           22,   12,   12,    7,  636,   11,   14,  240],
        [   2, 5390, 2562,  197,    6,  869,   63,    8,    2,  233,   50,   21,
            0,  608,  101,   12,   27,   62, 2609,  134],
        [   5,   36,    2,   25,    0,    5,    2, 8421,    4,    0,   10,   41,
            6, 5267,   15,   30,    5,    2,    0,    4],
        [1343,   69,  141,   46, 1214,    8, 2789, 4534,   41,   32,   20,  241,
         5653,    4,   46,    0,   67,    0,   39,    6],
        [  10,   30,  197,  135,   17,  230,  163, 3896,  372,   30,    5,  896,
            0,  119, 2260, 1624,  252,  749,   65,   14]])


배치 크기가 5이기 때문에 5개의 샘플이 출력되는 것을 볼 수 있습니다. 각 샘플의 길이는 20의 길이를 가지는데, 이는 앞서 초기에 필드를 정의할 때 fix_length를 20으로 정해주었기 때문입니다. 다시 말해 하나의 미니 배치의 크기는 (배치 크기 × fix_length)입니다.

In [11]:
# 미니 배치의 크기를 출력해봅시다.
print(batch.text.shape)

torch.Size([5, 20])


이제 batch_first를 False로 한 경우를 보겠습니다.

### 4. 필드 재정의하기(torchtext.data)

In [12]:
# 필드 정의
# TEXT 필드에서 batch_first = True 인자를 제거하였습니다. 기본값이 False이므로 batch_first의 값이 False가 됩니다.
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

In [13]:
from torchtext.data import TabularDataset
from torchtext.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠습니다.
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치

In [14]:
# 첫번째 미니 배치 출력
print(batch.text)

tensor([[  10,    2,    5, 1343,   10],
        [  20, 5390,   36,   69,   30],
        [ 469, 2562,    2,  141,  197],
        [ 137,  197,   25,   46,  135],
        [ 590,    6,    0, 1214,   17],
        [1450,  869,    5,    8,  230],
        [   4,   63,    2, 2789,  163],
        [ 197,    8, 8421, 4534, 3896],
        [ 605,    2,    4,   41,  372],
        [ 827,  233,    0,   32,   30],
        [   2,   50,   10,   20,    5],
        [ 670,   21,   41,  241,  896],
        [  22,    0,    6, 5653,    0],
        [  12,  608, 5267,    4,  119],
        [  12,  101,   15,   46, 2260],
        [   7,   12,   30,    0, 1624],
        [ 636,   27,    5,   67,  252],
        [  11,   62,    2,    0,  749],
        [  14, 2609,    0,   39,   65],
        [ 240,  134,    4,    6,   14]])


In [15]:
# 하나의 미니 배치의 크기는 (fix_length × 배치 크기)입니다.
print(batch.text.shape)

torch.Size([20, 5])
